In [ ]:
!wget https://dbdmg.polito.it/dbdmg_web/wp-content/uploads/2021/12/DSL2122_january_dataset.zip
!unzip DSL2122_january_dataset.zip

# Import required libraries

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

# Preview of Dataset

In [ ]:
dataframe = pd.read_csv('/content/DSL2122_january_dataset/development.csv')
dataframe

,sentiment,ids,date,flag,user,text
0,1,1833972543,Mon May 18 01:08:27 PDT 2009,NO_QUERY,Killandra,"@MissBianca76 Yes, talking helps a lot.. going..."
1,1,1980318193,Sun May 31 06:23:17 PDT 2009,NO_QUERY,IMlisacowan,SUNSHINE. livingg itttt. imma lie on the grass...
2,1,1994409198,Mon Jun 01 11:52:54 PDT 2009,NO_QUERY,yaseminx3,@PleaseBeMine Something for your iphone
3,0,1824749377,Sun May 17 02:45:34 PDT 2009,NO_QUERY,no_surprises,@GabrielSaporta couldn't get in to the after p...
4,0,2001199113,Tue Jun 02 00:08:07 PDT 2009,NO_QUERY,Rhi_ShortStack,@bradiewebbstack awww is andy being mean again...
...,...,...,...,...,...,...
224989,0,2261324310,Sat Jun 20 20:36:48 PDT 2009,NO_QUERY,CynthiaBuroughs,@Dropsofreign yeah I hope Iran people reach fr...
224990,1,1989408152,Mon Jun 01 01:25:45 PDT 2009,NO_QUERY,unitechy,Trying the qwerty keypad
224991,0,1991221316,Mon Jun 01 06:38:10 PDT 2009,NO_QUERY,Xaan,I love Jasper &amp; Jackson but that wig in th...
224992,0,2239702807,Fri Jun 19 08:51:56 PDT 2009,NO_QUERY,Ginger_Billie,I am really tired and bored and bleh! I feel c...


# Analyzing Dataset

In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224994 entries, 0 to 224993
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentiment  224994 non-null  int64 
 1   ids        224994 non-null  int64 
 2   date       224994 non-null  object
 3   flag       224994 non-null  object
 4   user       224994 non-null  object
 5   text       224994 non-null  object
dtypes: int64(2), object(4)
memory usage: 10.3+ MB


We need only Sentiment and Text

In [ ]:
""" Some Examples """

print(dataframe['text'][63])
# In this case one word is Binary(At least it is not english)

print(dataframe['text'][51])
# In this case we have @id + website

print(dataframe['text'][149])
# In this case word [-O] is not a word 


""" So many cases exist in the dataset like cases above and needs to be cleaned """

@Md8oz that hurt  ã?³ just fucked me harddddd lol
@HOTTVampChick here you go  http://bit.ly/qNtRH
[-O] Just got off work getting ready to leave for INDPLS 500 mini-marathon. Wish me luck.  http://tinyurl.com/cnn45k


' So many cases exist in the dataset like cases above and needs to be cleaned '

# Cleaning Data

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
import re 
noise_chars = ['#','?','!','+','*','{','}',"|",'(',')','$','%',';','_','-','~','<','>',',',':','=','[',']','\\','\t','\n']

texts = dataframe['text']
sentiments = dataframe['sentiment']

new_sentiments = []
new_texts = []
for i in range(len(texts)):
  text = texts[i]
  for c in noise_chars:
    text = text.replace(c,"")
  texts[i] = " ".join([word for word in text.split() if word not in stop_words]).lower()

for i in range(len(texts)):
  list_of_words = texts[i].split()
  list_of_noise = ['@','http','&','^','www']
  tmp = []
  for word in list_of_words[:]:
    if len(word) > 1 :
      has_noise = False
      for noise in list_of_noise:
        if noise in word:
          has_noise = True
          break
      if has_noise == False:
        tmp.append(word)
  if len(tmp) > 0:
    new_tweet = " ".join(tmp)
    new_texts.append(new_tweet)
    new_sentiments.append(sentiments[i])

print(len(new_texts))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


224017


In [ ]:
maxLen = 0
for text in new_texts:
  if len(text) > maxLen:
    maxLen = len(text)

print("Max lenght of a tweet has ",maxLen," words")

Max lenght of a tweet has  320  words


# Split Train and Validation

In [ ]:
train_x , val_x, train_y ,val_y = train_test_split(np.array(new_texts),np.array(new_sentiments),test_size = .15,stratify=np.array(new_sentiments))

print('train : ',train_x.shape , train_y.shape)
print('val : ',val_x.shape,val_y.shape)

train :  (190414,) (190414,)
val :  (33603,) (33603,)


# Apply tf-idf

In [ ]:
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(train_x)
tf_x_test = vectorizer.transform(val_x)

#  Support Vector Machine

In [ ]:
clf = LinearSVC(random_state=1)

clf.fit(tf_x_train,train_y)
y_test_pred=clf.predict(tf_x_test)
target_names = ['Negative', 'Positive']
print(classification_report(val_y, y_test_pred, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.74      0.69      0.72     14160
    Positive       0.79      0.82      0.80     19443

    accuracy                           0.77     33603
   macro avg       0.76      0.76      0.76     33603
weighted avg       0.77      0.77      0.77     33603



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(tf_x_train,train_y)
y_test_pred=clf.predict(tf_x_test)
target_names = ['Negative', 'Positive']
print(classification_report(val_y, y_test_pred, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.57      0.43      0.49     14160
    Positive       0.65      0.76      0.70     19443

    accuracy                           0.62     33603
   macro avg       0.61      0.60      0.59     33603
weighted avg       0.61      0.62      0.61     33603



# MLP Classifier

In [ ]:
clf = MLPClassifier(random_state=0,max_iter=100)
clf.fit(tf_x_train,train_y)
y_test_pred=clf.predict(tf_x_test)
target_names = ['Negative', 'Positive']
print(classification_report(val_y, y_test_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

    Negative       0.73      0.71      0.72     14160
    Positive       0.79      0.80      0.80     19443

    accuracy                           0.76     33603
   macro avg       0.76      0.76      0.76     33603
weighted avg       0.76      0.76      0.76     33603



# Decision Tree Classifier

In [ ]:
params = {
    "max_depth": [None, 2, 3, 4, 5],
    "min_impurity_decrease": [0, .01, .03, .07, .09, .11]
}

accuracies = []
for config in ParameterGrid(params):
    clf = DecisionTreeClassifier(**config)
    clf.fit(tf_x_train, train_y)
    accuracies.append(accuracy_score(val_y, clf.predict(tf_x_test)))
print(max(accuracies))

0.7035383745498914


In [ ]:
best_config = list(ParameterGrid(params))[np.argmax(accuracies)]
clf = DecisionTreeClassifier(**best_config)
clf.fit(tf_x_train, train_y)
print(classification_report(val_y, clf.predict(tf_x_test), target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.65      0.64      0.65     14160
    Positive       0.74      0.75      0.75     19443

    accuracy                           0.70     33603
   macro avg       0.70      0.70      0.70     33603
weighted avg       0.70      0.70      0.70     33603



# Random Forest Classifier

In [ ]:
from scipy.stats import mode

class MyRandomForestClassifier:
    def __init__(self, n_estimators=10, max_features='sqrt'):
        self.trees = [ DecisionTreeClassifier(max_features=max_features)\
                       for _ in range(n_estimators) ]
    
    def fit(self, X, y):
        for tree in self.trees:
            subset = np.random.choice(range(X.shape[0]),\
                                      size=X.shape[0],\
                                      replace=True)
            tree.fit(X[subset], y[subset])
    
    def predict(self, X):
        # get predictions of all trees for X 
        predictions = [ tree.predict(X) for tree in self.trees ] 
        return mode(predictions, axis=0)[0][0]

In [ ]:
clf = MyRandomForestClassifier(10, 28)
clf.fit(tf_x_train,train_y)
y_test_pred=clf.predict(tf_x_test)
target_names = ['Negative', 'Positive']
print(classification_report(val_y, y_test_pred, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.67      0.69      0.68     14160
    Positive       0.77      0.75      0.76     19443

    accuracy                           0.73     33603
   macro avg       0.72      0.72      0.72     33603
weighted avg       0.73      0.73      0.73     33603

